In [1]:
import os
import json
import requests
import sys

# Ensure local project directory is prioritized
sys.path.insert(0, os.path.abspath("injections"))

from Plunger_helper import raw_json
# import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  
# --- LLM Setup ---
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)




In [2]:
from collections import defaultdict

def safe_format(template, **kwargs):
    return template.format_map(defaultdict(lambda: "", kwargs))


In [3]:
import re
import json

def clean_markdown_json(text):
    # Remove triple backticks + optional language tag
    cleaned = re.sub(r"^```(?:json)?\n", "", text)
    cleaned = re.sub(r"\n```$", "", cleaned)
    return cleaned.strip()

In [5]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fastapi]m1/2 [fastapi]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def get_chat_completion(prompt, model) -> str:
    response = model.invoke(prompt)
    return response.content.strip()

def execute_sql(sql: str) -> dict:
    print("\n Sending SQL to server:")
    print(sql)
    
    response = requests.post("http://localhost:8765/", data=sql)
    print(" Raw Response:", response.status_code, response.text)
    response.raise_for_status()
    return response.json()


def main():
    user_query = input("Enter your plunger lift question: ")
    
    event_context = raw_json["PromptInjection"]

    decomposer_prompt = f"""
You are a task decomposer. Break down the user's natural language question into a structured query that highlights what the user wants to know, what metrics or filters are involved, and what kind of result is expected.

## User Query:
{user_query}

Return a structured breakdown only.
""".strip()

    print("\nDecomposing user query...")
    structured_query = get_chat_completion(decomposer_prompt, llm)
    print("\nDecomposed Query:\n", structured_query.strip())
    print("\n user_query", user_query)
    sql_prompt = f"""
You are an expert SQL generator. Given a structured query and schema context, return a valid SQL query for an SQLite database. Do not explain anything.

## Structured Query:
{structured_query}

## Schema and Event Hierarchy:
{event_context}

ONLY return the SQL query.
""".strip()

    print("\nSending SQL generation prompt to LLM...")
    sql = get_chat_completion(sql_prompt, llm)

    sql = sql.strip().strip("```sql").strip("```").strip()
    sql = sql.rstrip(";") + ";"

    print("\nSQL Generated:\n", sql)

    try:
        result = execute_sql(sql)
        print("\nRaw Query Result:")
        from pprint import pprint
        pprint(result)
    except Exception as e:
        print("\nSQL execution failed:", str(e))
        return

    summary_prompt = f"""
You are a data analyst. Based on the following SQL query, its result, and the user’s original question, generate a clear and concise explanation that helps a human understand the insight in plain English.

## User Query:
{user_query}

## SQL Query:
{sql}

## Query Result:
{result}

Write a friendly, human-readable summary.
""".strip()

    print("\nGenerating human-friendly summary...")
    final_summary = get_chat_completion(summary_prompt, llm)
    print("\nFinal Answer:\n", final_summary.strip())


if __name__ == "__main__":
    main()


Decomposing user query...

Decomposed Query:
 **Query Breakdown**

* **Objective**: Identify a specific cycle
* **Condition**: Plunger did not arrive
* **Metrics/Filters**: 
	+ Item: Plunger
	+ Status: Did not arrive
* **Expected Result**: A specific cycle where the plunger did not arrive

 user_query show me the cycle where plunger did not arrive

Sending SQL generation prompt to LLM...

SQL Generated:
 SELECT e.cycle_id 
FROM EVENTS e 
JOIN PLUNGER_ARRIVAL_STATUS_EVENTS p 
ON e.plunger_arrival_status_event = p.rowid 
WHERE p.non_arrival = 1;

 Sending SQL to server:
SELECT e.cycle_id 
FROM EVENTS e 
JOIN PLUNGER_ARRIVAL_STATUS_EVENTS p 
ON e.plunger_arrival_status_event = p.rowid 
WHERE p.non_arrival = 1;
 Raw Response: 200 [{"cycle_id": 121}, {"cycle_id": 217}, {"cycle_id": 218}, {"cycle_id": 458}, {"cycle_id": 459}, {"cycle_id": 461}, {"cycle_id": 462}, {"cycle_id": 468}, {"cycle_id": 469}, {"cycle_id": 121}, {"cycle_id": 217}, {"cycle_id": 218}, {"cycle_id": 458}, {"cycle_id": 45

In [22]:
# def get_chat_completion(prompt, model) -> str:
#     response = model.invoke(prompt)
#     return response.content.strip()

# def execute_sql(sql: str) -> dict:
#     print("\n Sending SQL to server:")
#     print(sql)
    
#     response = requests.post("http://localhost:8765/", data=sql)
#     print(" Raw Response:", response.status_code, response.text)
#     response.raise_for_status()
#     return response.json()


# def load_prompt(file_path):
#     with open(file_path, 'r') as f:
#         return f.read().strip()


# def main():
#     user_query = input("Enter your plunger lift question: ")

#     # Example: non-nested key access with error handling
#     event_context1 = raw_json.get("schema", "Default schema value or handle missing key")
#     event_context2 = raw_json.get("PromptInjection1", "Default PromptInjection1 value or handle missing key")
#     event_context3 = raw_json.get("event_details", "Default event_details value or handle missing key")

#     # Load prompt templates
#     decomposer_template = load_prompt("prompts/decomposer.md")
#     sql_template = load_prompt("prompts/sqlquery.md")
#     summary_template = load_prompt("prompts/final_analyst.md")

#     # Format decomposer prompt
#     decomposer_prompt = decomposer_template.format(user_query=user_query)

#     print("\nDecomposing user query...")
#     structured_query = get_chat_completion(decomposer_prompt, llm)
#     print("\nDecomposed Query:\n", structured_query.strip())

#     # Format SQL generation prompt
#     sql_prompt = safe_format(
#         sql_template,
#         structured_query=structured_query.strip(),
#         schema=event_context1,
#         PromptInjection1=event_context2,
#         event_details=event_context3
#     )



#     print("\nSending SQL generation prompt to LLM...")
#     sql = get_chat_completion(sql_prompt, llm)

#     sql = sql.strip().strip("```sql").strip("```").strip()
#     sql = sql.rstrip(";") + ";"

#     print("\nSQL Generated:\n", sql)

#     try:
#         result = execute_sql(sql)
#         print("\nRaw Query Result:")
#         from pprint import pprint
#         pprint(result)
#     except Exception as e:
#         print("\nSQL execution failed:", str(e))
#         return

#     # Format summary prompt
#     summary_prompt = summary_template.format(
#         user_query=user_query,
#         sql=sql,
#         result=result
#     )

#     print("\nGenerating human-friendly summary...")
#     final_summary = get_chat_completion(summary_prompt, llm)
#     print("\nFinal Answer:\n", final_summary.strip())


# if __name__ == "__main__":
#     main()
